In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
import utils.plot as pu

import sympy
from sympy import Symbol, symbols, RR
import numpy as np
from mpmath import *
from matplotlib import pyplot as plt
import scipy

In [28]:
n = 3 # > 1, number of shells

In [29]:
# define the symbols

pi = Symbol("pi")
R, K, M, rho = [None], [Symbol('K0', real=True, positive=True)], [None], [None]
for i in range(1, n + 1):
    R.append(Symbol(f"R{i}", real=True, positive=True))
    K.append(Symbol(f"K{i}", real=True, positive=True))
    M.append(Symbol(f"M{i}", real=True, positive=True))
    rho.append(Symbol(f"rho{i}", real=True, positive=True))

In [30]:
# recursively construct the polynomial

M[1] = 4 * pi * rho[1] * R[1]**2 * K[0]**5
K[1] = K[0] * (1 - 2 * pi * rho[1] * R[1] * K[0]**4)

for i in range(2, n + 1):
    M[i] = M[i-1] + 4 * pi * rho[i] * R[i]**2 * (K[i-1] + M[i-1]/(2*R[i]))**5
    K[i] = K[i-1] - 2 * pi * rho[i] * R[i] * (K[i-1] + M[i-1]/(2*R[i]))**5

# substitute into the master equation
master = 1 - K[n-1] + (M[n] - M[n-1]) / (2 * R[n])

In [31]:
master

-K0*(-2*K0**4*R1*pi*rho1 + 1) + 2*R2*pi*rho2*(2*K0**5*R1**2*pi*rho1/R2 + K0*(-2*K0**4*R1*pi*rho1 + 1))**5 + 2*R3*pi*rho3*(K0*(-2*K0**4*R1*pi*rho1 + 1) - 2*R2*pi*rho2*(2*K0**5*R1**2*pi*rho1/R2 + K0*(-2*K0**4*R1*pi*rho1 + 1))**5 + (4*K0**5*R1**2*pi*rho1 + 4*R2**2*pi*rho2*(2*K0**5*R1**2*pi*rho1/R2 + K0*(-2*K0**4*R1*pi*rho1 + 1))**5)/(2*R3))**5 + 1

In [32]:
# define symbols for convenient substitution to simplify the polynomial

l, k, kappa = [None], [None], [None]

for i in range(1, n + 1):
    l.append(Symbol(f"l{i}"))
    k.append(Symbol(f"k{i}"))
    kappa.append(Symbol(f"kappa{i}"))

l, k, kappa

([None, l1, l2, l3], [None, k1, k2, k3], [None, kappa1, kappa2, kappa3])

In [33]:
_master = master.copy()

# substitute k[i] := 4 * pi * R[i]**2
for i in range(1, n + 1):
    expr = k[i] / (4 * pi * R[i]**2)
    _master = _master.subs(rho[i], expr)

_master

-K0*(-K0**4*k1/(2*R1) + 1) + 1 + k3*(K0*(-K0**4*k1/(2*R1) + 1) + (K0**5*k1 + k2*(K0**5*k1/(2*R2) + K0*(-K0**4*k1/(2*R1) + 1))**5)/(2*R3) - k2*(K0**5*k1/(2*R2) + K0*(-K0**4*k1/(2*R1) + 1))**5/(2*R2))**5/(2*R3) + k2*(K0**5*k1/(2*R2) + K0*(-K0**4*k1/(2*R1) + 1))**5/(2*R2)

In [34]:
# substitute l[i] := k[i] / (2 *R[i])
for i in range(1 , n + 1):
    _master = _master.subs(k[i], l[i]*2*R[i])

_master

-K0*(-K0**4*l1 + 1) + l2*(K0**5*R1*l1/R2 + K0*(-K0**4*l1 + 1))**5 + l3*(K0*(-K0**4*l1 + 1) - l2*(K0**5*R1*l1/R2 + K0*(-K0**4*l1 + 1))**5 + (2*K0**5*R1*l1 + 2*R2*l2*(K0**5*R1*l1/R2 + K0*(-K0**4*l1 + 1))**5)/(2*R3))**5 + 1

In [36]:
# substitute kappa[i] := R[i-1] / R[i]
for i in range(2, n + 1):
    _master = _master.subs(R[i-1], R[i] * kappa[i-1])

_master.simplify()

K0**5*l1 + K0**5*l2*(K0**4*kappa1*l1 - K0**4*l1 + 1)**5 - K0**5*l3*(-K0**4*kappa1*kappa2*l1 - K0**4*kappa2*l2*(K0**4*kappa1*l1 - K0**4*l1 + 1)**5 + K0**4*l1 + K0**4*l2*(K0**4*kappa1*l1 - K0**4*l1 + 1)**5 - 1)**5 - K0 + 1

In [19]:
# only for n = 2
hand_written_master = 1 - K[0] + l[1] * K[0]**5 + l[2] * (K[0] - l[1] * (1 - kappa[1]) * K[0]**5)**5

# verify that the symbolic master polynomial matches the hand‑derived expression
sympy.simplify(_master - hand_written_master) == 0 

True